### EDA
Created variables defining appointment history for patients and performed EDA. Output several datasets with different appointments for later model exploration.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import calendar
import pickle

%matplotlib inline
plt.style.use('seaborn-pastel')

import seaborn as sns
sns.set(style="whitegrid")

%run EDA_functions.py

In [23]:
with open('pickles/appt.pickle','rb') as read_file:
    a_df = pickle.load(read_file)

### Create alternative dataset that only contains appointments with historical data

In [24]:
a_df.head(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,SchedDay,SchedTime,SchedDayofWeek,ApptDay,ApptDayofWeek,DayDiff,Handcap_bin,DayDiff2,Age2,Neighbourhood2
0,9.598513e+13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,...,2016-04-27,08:36:51,Wednesday,2016-04-29,Friday,2,0,2,76,REPÚBLICA
1,7.336882e+14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,...,2016-04-27,15:05:12,Wednesday,2016-04-29,Friday,2,0,2,23,GOIABEIRAS
2,3.449833e+12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,...,2016-04-27,15:39:58,Wednesday,2016-04-29,Friday,2,0,2,39,GOIABEIRAS
3,7.812456e+13,5629123,F,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,CONQUISTA,0,0,0,...,2016-04-27,12:48:25,Wednesday,2016-04-29,Friday,2,0,2,19,CONQUISTA
4,7.345362e+14,5630213,F,2016-04-27T14:58:11Z,2016-04-29T00:00:00Z,30,NOVA PALESTINA,0,0,0,...,2016-04-27,14:58:11,Wednesday,2016-04-29,Friday,2,0,2,30,NOVA PALESTINA


In [47]:
a_df2 = a_df.copy()
a_df2.sort_values(['PatientId','ApptDay'],inplace=True)
#days elapsed since prior appointment
a_df2['ApptDayDiff'] = a_df2.groupby(['PatientId'])[['ApptDay']].diff()
#date of prior appointment
a_df2['PrevApptDay'] = a_df2.groupby(['PatientId'])[['ApptDay']].shift()
#missed prior appointment
a_df2['PrevAppt_No_show'] = a_df2.groupby(['PatientId'])[['No_show_num']].shift()
#days elapsed between prior appt and current scheduled appt
a_df2['PrevAppt_Sched_diff'] = a_df2['SchedDay'] - a_df2['PrevApptDay']

In [48]:
#proportion of prior appointments that had been missed (rolling proportion)
a_df2['appt_count'] = 1
a_df2['total_appt_count'] = a_df2.groupby(['PatientId'])['appt_count'].cumsum()
a_df2['total_no_show_count'] = a_df2.groupby(['PatientId'])['No_show_num'].cumsum()
a_df2['no_show_rate'] = a_df2['total_no_show_count']/a_df2['total_appt_count']
a_df2['no_show_rate2'] = a_df2.groupby(['PatientId'])['no_show_rate'].shift() # rate of prior appointments
a_df2['prior_appt_count'] = a_df2.groupby(['PatientId'])['total_appt_count'].shift() # number of prior appointments
a_df2['prior_no_show_count'] = a_df2.groupby(['PatientId'])['total_no_show_count'].shift() # number of prior appointments

In [49]:
a_df2[['PatientId','ApptDay','ApptDayDiff','No_show_num','total_appt_count','total_no_show_count','prior_appt_count','prior_no_show_count','no_show_rate','no_show_rate2']][19:35]

,PatientId,ApptDay,ApptDayDiff,No_show_num,total_appt_count,total_no_show_count,prior_appt_count,prior_no_show_count,no_show_rate,no_show_rate2
19174,142133299.0,2016-05-16,NaT,1,1,1,NaN,NaN,1.000000,NaN
67501,142133299.0,2016-06-07,22 days,0,2,1,1.0,1.0,0.500000,1.0
29793,178168233.0,2016-05-02,NaT,0,1,0,NaN,NaN,0.000000,NaN
18327,179874775.0,2016-05-24,NaT,1,1,1,NaN,NaN,1.000000,NaN
46038,188964456.0,2016-05-18,NaT,0,1,0,NaN,NaN,0.000000,NaN
58107,211124677.0,2016-06-02,NaT,0,1,0,NaN,NaN,0.000000,NaN
58342,211124677.0,2016-06-08,6 days,0,2,0,1.0,0.0,0.000000,0.0
44897,215778678.0,2016-05-02,NaT,1,1,1,NaN,NaN,1.000000,NaN
29729,243921212.0,2016-05-03,NaT,0,1,0,NaN,NaN,0.000000,NaN
32867,251541539.0,2016-05-13,NaT,1,1,1,NaN,NaN,1.000000,NaN


In [50]:
a_df2.shape

(71163, 36)

In [51]:
def resched(x):
    if x['PrevAppt_No_show'] == 1 and x['PrevAppt_Sched_diff'] == dt.timedelta(0):
        return 1
    else:
        return 0
    
def followup(x):
    if x['PrevAppt_No_show'] == 0 and x['PrevAppt_Sched_diff'] == dt.timedelta(0):
        return 1
    else:
        return 0

In [52]:
# categorizing the visit types
# a true "reschedule" - they missed their prior appointment and rescheduled it on the same day as their prior appointment
a_df2['Reschedule'] = a_df2.apply(resched,axis=1)
# a "follow-up" - they attended their prior appointment and they scheduled the current one on the same day
a_df2['Follow_up'] = a_df2.apply(followup,axis=1)

In [53]:
a_df2['ApptDayDiff'] = [x.days for x in a_df2['ApptDayDiff']]
a_df2['PrevAppt_Sched_diff'] = [x.days for x in a_df2['PrevAppt_Sched_diff']]

In [54]:
# identify all patients with more than 1 appointment
a_df2a = a_df2.groupby(['PatientId'],as_index=False)[['No_show']].count()
a_df2b = a_df2a.loc[a_df2a['No_show']>1]
print(a_df2b.shape)
a_df2b.head(5)

(14531, 2)


,PatientId,No_show
4,22638656.0,2
8,64851211.0,2
14,122451254.0,2
16,142133299.0,2
20,211124677.0,2


In [55]:
# keep only appointments for patients with more than 1 appointment
a_df3 = a_df2.merge(a_df2b['PatientId'],how='inner',left_on = 'PatientId',right_on = 'PatientId')
print(a_df3.shape)
a_df3.head(10)

(39984, 38)


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,PrevAppt_Sched_diff,appt_count,total_appt_count,total_no_show_count,no_show_rate,no_show_rate2,prior_appt_count,prior_no_show_count,Reschedule,Follow_up
0,22638656.0,5580835,F,2016-04-14T07:23:30Z,2016-05-03T00:00:00Z,22,INHANGUETÁ,0,0,0,...,NaN,1,1,0,0.0,NaN,NaN,NaN,0,0
1,22638656.0,5715081,F,2016-05-18T13:37:12Z,2016-06-08T00:00:00Z,23,INHANGUETÁ,0,0,0,...,15.0,1,2,0,0.0,0.0,1.0,0.0,0,0
2,64851211.0,5683383,F,2016-05-11T07:28:23Z,2016-05-13T00:00:00Z,29,MARUÍPE,0,0,0,...,NaN,1,1,1,1.0,NaN,NaN,NaN,0,0
3,64851211.0,5697532,F,2016-05-13T16:22:26Z,2016-05-17T00:00:00Z,29,MARUÍPE,0,0,0,...,0.0,1,2,1,0.5,1.0,1.0,1.0,1,0
4,122451254.0,5556646,M,2016-04-07T12:39:53Z,2016-05-02T00:00:00Z,28,JESUS DE NAZARETH,0,0,0,...,NaN,1,1,1,1.0,NaN,NaN,NaN,0,0
5,122451254.0,5576521,M,2016-04-13T08:35:28Z,2016-05-09T00:00:00Z,28,JESUS DE NAZARETH,0,0,0,...,-19.0,1,2,2,1.0,1.0,1.0,1.0,0,0
6,142133299.0,5683430,F,2016-05-11T07:31:01Z,2016-05-16T00:00:00Z,80,ARIOVALDO FAVALESSA,0,1,0,...,NaN,1,1,1,1.0,NaN,NaN,NaN,0,0
7,142133299.0,5767863,F,2016-06-03T07:09:49Z,2016-06-07T00:00:00Z,80,ARIOVALDO FAVALESSA,0,1,0,...,18.0,1,2,1,0.5,1.0,1.0,1.0,0,0
8,211124677.0,5741812,M,2016-05-30T07:27:04Z,2016-06-02T00:00:00Z,56,SANTOS REIS,0,0,0,...,NaN,1,1,0,0.0,NaN,NaN,NaN,0,0
9,211124677.0,5761936,M,2016-06-02T07:13:31Z,2016-06-08T00:00:00Z,56,SANTOS REIS,0,0,0,...,0.0,1,2,0,0.0,0.0,1.0,0.0,0,1


In [56]:
#drop the first appointments
a_df4 = a_df3.loc[a_df3['PrevApptDay'].isna() == False]

In [57]:
a_df4.shape

(25453, 38)

In [58]:
a_df4.head(10)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,PrevAppt_Sched_diff,appt_count,total_appt_count,total_no_show_count,no_show_rate,no_show_rate2,prior_appt_count,prior_no_show_count,Reschedule,Follow_up
1,22638656.0,5715081,F,2016-05-18T13:37:12Z,2016-06-08T00:00:00Z,23,INHANGUETÁ,0,0,0,...,15.0,1,2,0,0.000000,0.0,1.0,0.0,0,0
3,64851211.0,5697532,F,2016-05-13T16:22:26Z,2016-05-17T00:00:00Z,29,MARUÍPE,0,0,0,...,0.0,1,2,1,0.500000,1.0,1.0,1.0,1,0
5,122451254.0,5576521,M,2016-04-13T08:35:28Z,2016-05-09T00:00:00Z,28,JESUS DE NAZARETH,0,0,0,...,-19.0,1,2,2,1.000000,1.0,1.0,1.0,0,0
7,142133299.0,5767863,F,2016-06-03T07:09:49Z,2016-06-07T00:00:00Z,80,ARIOVALDO FAVALESSA,0,1,0,...,18.0,1,2,1,0.500000,1.0,1.0,1.0,0,0
9,211124677.0,5761936,M,2016-06-02T07:13:31Z,2016-06-08T00:00:00Z,56,SANTOS REIS,0,0,0,...,0.0,1,2,0,0.000000,0.0,1.0,0.0,0,1
11,251541539.0,5692489,F,2016-05-12T15:29:02Z,2016-05-19T00:00:00Z,85,MARUÍPE,0,1,0,...,-1.0,1,2,1,0.500000,1.0,1.0,1.0,0,0
12,251541539.0,5721140,F,2016-05-19T15:31:13Z,2016-06-01T00:00:00Z,85,MARUÍPE,0,1,0,...,0.0,1,3,1,0.333333,0.5,2.0,1.0,0,1
14,355744126.0,5732684,M,2016-05-24T12:13:17Z,2016-06-03T00:00:00Z,81,TABUAZEIRO,0,0,0,...,-8.0,1,2,1,0.500000,0.0,1.0,0.0,0,0
16,422861257.0,5707143,F,2016-05-17T09:13:43Z,2016-05-19T00:00:00Z,30,CRUZAMENTO,1,0,0,...,15.0,1,2,0,0.000000,0.0,1.0,0.0,0,0
17,422861257.0,5779051,F,2016-06-06T17:37:40Z,2016-06-08T00:00:00Z,30,CRUZAMENTO,1,0,0,...,18.0,1,3,0,0.000000,0.0,2.0,0.0,0,0


In [59]:
test = a_df4[['no_show_rate2','Reschedule']]
test.drop_duplicates()
test.loc[(test['no_show_rate2'] < 0) & (test['Reschedule'] == 1)]

,no_show_rate2,Reschedule


In [60]:
with open('pickles/appt_v2_all.pickle', 'wb') as to_write:
    pickle.dump(a_df2, to_write)

In [61]:
with open('pickles/appt_v2.pickle', 'wb') as to_write:
    pickle.dump(a_df4, to_write)

### Create alternative dataset that only contains one-time appointments

In [40]:
# identify all patients with just 1 appointment
a_df2c = a_df2a.loc[a_df2a['No_show']==1]
print(a_df2c.shape)
a_df2c.head(5)

(31179, 2)


,PatientId,No_show
0,3.921784e+04,1
1,1.417242e+05,1
2,5.376153e+05,1
3,5.628261e+06,1
5,5.216894e+07,1


In [41]:
# keep only appointments for patients with 1 appointment
a_df5 = a_df2.merge(a_df2c['PatientId'],how='inner',left_on = 'PatientId',right_on = 'PatientId')
a_df5.drop(columns=['ApptDayDiff','PrevApptDay','PrevAppt_No_show','PrevAppt_Sched_diff'],inplace=True)
print(a_df5.shape)
a_df5.head(10)

(31179, 34)


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,Neighbourhood2,appt_count,total_appt_count,total_no_show_count,no_show_rate,no_show_rate2,prior_appt_count,prior_no_show_count,Reschedule,Follow_up
0,3.921784e+04,5751990,F,2016-05-31T10:56:41Z,2016-06-03T00:00:00Z,44,PRAIA DO SUÁ,0,0,0,...,PRAIA DO SUÁ,1,1,0,0.0,NaN,NaN,NaN,0,0
1,1.417242e+05,5637648,M,2016-04-29T07:13:36Z,2016-05-02T00:00:00Z,12,FORTE SÃO JOÃO,0,0,0,...,FORTE SÃO JOÃO,1,1,0,0.0,NaN,NaN,NaN,0,0
2,5.376153e+05,5637728,F,2016-04-29T07:19:57Z,2016-05-06T00:00:00Z,14,FORTE SÃO JOÃO,0,0,0,...,FORTE SÃO JOÃO,1,1,0,0.0,NaN,NaN,NaN,0,0
3,5.628261e+06,5680449,M,2016-05-10T11:58:18Z,2016-05-13T00:00:00Z,13,PARQUE MOSCOSO,0,0,0,...,PARQUE MOSCOSO,1,1,1,1.0,NaN,NaN,NaN,0,0
4,5.216894e+07,5607220,F,2016-04-20T11:22:15Z,2016-05-17T00:00:00Z,28,JARDIM DA PENHA,0,0,0,...,JARDIM DA PENHA,1,1,0,0.0,NaN,NaN,NaN,0,0
5,5.422400e+07,5613714,M,2016-04-25T09:36:18Z,2016-05-11T00:00:00Z,32,ITARARÉ,0,0,0,...,ITARARÉ,1,1,0,0.0,NaN,NaN,NaN,0,0
6,6.249793e+07,5671723,M,2016-05-09T07:05:33Z,2016-05-11T00:00:00Z,10,JARDIM DA PENHA,0,0,0,...,JARDIM DA PENHA,1,1,0,0.0,NaN,NaN,NaN,0,0
7,7.838548e+07,5640016,F,2016-04-29T09:59:11Z,2016-05-02T00:00:00Z,21,CARATOÍRA,0,0,0,...,CARATOÍRA,1,1,0,0.0,NaN,NaN,NaN,0,0
8,7.922850e+07,5743266,F,2016-05-30T08:51:07Z,2016-06-08T00:00:00Z,21,NOVA PALESTINA,0,0,0,...,NOVA PALESTINA,1,1,0,0.0,NaN,NaN,NaN,0,0
9,8.658474e+07,5672306,M,2016-05-09T07:54:13Z,2016-05-13T00:00:00Z,68,CARATOÍRA,0,0,0,...,CARATOÍRA,1,1,0,0.0,NaN,NaN,NaN,0,0


In [42]:
a_df5['No_show_num'].value_counts()

0    22333
1     8846
Name: No_show_num, dtype: int64